In [71]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


import random

print(tf.config.list_physical_devices('GPU'))
df = pd.read_csv("prep_data/image_data.csv")
df.head()

[]


,filename,scalar_x,scalar_y,s_button
0,7000.jpg,0,60,0
1,7001.jpg,0,87,0
2,7002.jpg,0,94,0
3,7003.jpg,0,105,0
4,7004.jpg,0,110,0


In [72]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("prep_data/image_data.csv")
    shuffled_df = df.sample(len(df))
    # 
    print(shuffled_df.head)  # Shuffled DataFrame
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"prep_data/{entry[0]}")

        prepared_y = np.delete(entry,0)

        X.append(entry_img)
        y.append(prepared_y)
    y = np.asarray(y).astype(np.float32)

    return X, y


In [73]:
X, y = create_input()



<bound method NDFrame.head of      filename  scalar_x  scalar_y  s_button
39   7039.jpg       338         0         0
83   7083.jpg       360         0         0
47   7047.jpg       340       325         0
15   7015.jpg         0       148         0
174  7174.jpg       133       202       100
..        ...       ...       ...       ...
163  7163.jpg       127       130       100
103  7103.jpg       369       274         0
26   7026.jpg         0       222         0
76   7076.jpg       356       118         0
41   7041.jpg       339       106         0

[222 rows x 4 columns]>


In [74]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer=Adam())

    return model



In [68]:
def train_model(model,X,y, epochs_req):

    train_split = int(len(y) * 0.8)
    X_train, X_test = np.array(X[:train_split - 1]), np.array(X[train_split:])
    y_train, y_test = y[:train_split - 1], y[train_split:]
    model.fit(X_train, y_train, epochs=(epochs_req), validation_data=(X_test, y_test))

    for i in range(20):
        tbp = np.asarray(X_test[i])
        tbp = np.expand_dims(tbp, axis=0)
        predictions = model.predict(tbp)
        print("pred", predictions, "real", y_test[i])


    return model

In [75]:
model = compile_model()

In [76]:
model = train_model(model, X, y, 2)

Epoch 1/2
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 1246451.8750 - val_loss: 75926.0703
Epoch 2/2
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 73688.8828 - val_loss: 24460.6465
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
pred [[ 45.85293   58.171085 130.38571 ]] real [145.  87. 100.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
pred [[ 42.45029   74.429115 181.31218 ]] real [126. 161. 100.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
pred [[ 52.786564  58.98493  129.18903 ]] real [342. 247.   0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
pred [[ 55.602253  70.67035  173.41788 ]] real [154.  62. 100.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
pred [[ 1.6125449 92.96576   99.97382  ]] real [153.  83. 100.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
pred [[ 33.626827  57.685356 122.701256]] real [146. 209. 100.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
pred [[ 70.430374  53.04464  123.54145 ]] real [  0. 158.   0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
pred [[ 60.176872  57.318478 116.48442 ]] real [132. 167. 100.]
1/1 ━━

In [77]:
model.save('cemetery.h5')

In [ ]:

model_rap = tf.keras.models.load_model('C:\\Users\\platy_move\\Documents\\GitHub\\neuralnet_skeleton\\cemetery.h5')
model_rap.summary()

model_rap.compile(loss='mean_squared_error', optimizer=Adam())
train_model(model_rap, X, y, 5)
model.save('/cemetery.h5')

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ max_pooling2d_12 (MaxPooling2D) │ (None, 180, 320, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 178, 318, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 89, 159, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 905664)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │    57,962,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,972,869 (221.15 MB)

 Trainable params: 57,972,867 (221.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 274107.0625 - val_loss: 150369.8281
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 81762.0625 - val_loss: 17788.9492
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 20568.0176 - val_loss: 13915.1250
Epoch 4/5
5/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 14226.4473

In [ ]:
# from google.colab.patches import cv2_imshow
# def draw_dot(image,button, x, y, radius=20, thickness=-1, color = None):
#     if color == None:
#       color = (255, 0, 0) if button == "left" else (0, 0, 255)
#     else:
#       color = (255, 255, 0) if button == 0 else (0, 255, 255)
#     cv2.circle(image, (x, y), radius, color, thickness)
# 
# df = pd.read_csv("/content/drive/MyDrive/prep_data/image_data.csv")
# ss = 100
# for i in range(20):
#   ss = ss + i
#   df_entry = df.loc[ss]
#   print(df_entry, type(df_entry))
# 
# 
#   img = cv2.imread(f"/content/drive/MyDrive/prep_data/{ss}.jpg")
#   tbp = np.asarray(img)
#   tbp = np.expand_dims(tbp, axis=0)
#   Y = model.predict(tbp)
#   print(Y)
#   pred_x, pred_y, pred_button = Y[0][0], Y[0][1], Y[0][2]
#   pred_x, pred_y, pred_button = round(pred_x*640), round(pred_y*360), round(pred_button)
#   print(pred_x, pred_y, pred_button)
# 
#   draw_dot(img, pred_button, pred_x, pred_y, color = pred_button)
#   cv2_imshow(img)

